In [18]:
import pandas as pd
import numpy as np
from statsmodels.tsa.arima_model import ARIMA
import matplotlib.pyplot as plt
import itertools
import warnings
warnings.filterwarnings("ignore")

import requests
from bs4 import BeautifulSoup as bs
from datetime import date,timedelta
from datetime import datetime as d

import plotly.graph_objects as go

In [19]:
import time
# from sklearn.externals import joblib
import joblib

from main_app import fit_model

In [20]:
from main_app import fit_model, forecast_cases
from Update_Database import update_database

In [2]:
# from main_app_v2 import fit_model, forecast_cases
from Update_Database_v2 import update_database

In [21]:
from main_app_v2 import fit_model, forecast_cases, get_previous_cases

In [149]:
today, yesterday, as_on = get_previous_cases("World")

In [150]:
yesterday

16114449

In [153]:
%%writefile app_v2.py
from main_app_v2 import fit_model, forecast_cases, get_previous_cases
from Update_Database import update_database
from website_elements_v2 import get_menu, embbed_graph, add_cases_compare
from datetime import date, timedelta
from datetime import datetime as d
from pandas import date_range
# import pandas as pd
# import plotly.graph_objects as go
import flask
from flask import request
app = flask.Flask(__name__)
app.config["DEBUG"] = True
from flask_cors import CORS


def result_to_html(result, region):
    today, yesterday, as_on = get_previous_cases(region)
    as_on = d.strptime(date.today().strftime('%Y-%m-%d'), "%Y-%m-%d").date()
    res=get_menu()
    res+='<div style="width: 100%; padding: 30px 0px 0px 0px;">\n'
    res+='<div class="container" style="display: flex; height: 600px;">\n'
    res+='<div style="width: 20%; padding: 10px 10px 10px 10px;" class="temp_1">\n'
    res+='<h5 style="font-size: 14px;"><strong style="font-size: 20px;"> In {} </strong> <br> as on : {}</h5>\n'.format(region, as_on)
    res+='<table id="t01">\n'
    yesterday_d = (as_on-timedelta(days=1)).strftime('%Y-%m-%d')
    dates = date_range(yesterday_d, periods=7).strftime('%Y-%m-%d')
    result = zip(dates,result)
    res+='<thead>\n'
    res+='<style> table, th, td {border: 2px solid black; border-collapse: collapse;} </style>'
    res+='<tr><th style="width: 40%; padding: 10px 10px 10px 10px;"><h3 style="text-align: center;"> Date </h3></th><th style="padding: 10px 10px 10px 10px;"><h3 style="text-align: center;">Predicted No. of Cases &#177;&nbsp; 5%</h3></th></tr>\n'
    res+='<tbody>'
    res+='<style> table, th, td {border: 2px solid black; border-collapse: collapse;} </style>' #padding-left:20px; padding-right:20px;} </style>'
    for row in result:
        res+='<tr><td style="padding-left:0px; padding-right:0px;"><h3 style="text-align: center;">&nbsp; {} &nbsp;</h3></td> <td style="padding-left:10px; padding-right:10px;"><h3 style="text-align: center;"> {} </h3></td></tr>\n'.format(row[0],row[1])
    res+='</tbody>'
    res+='</table>\n\n'
    res+='<p style="font-size: 14px; line-height: 1.8;"><strong>Confirmed Cases for Yesterday : {}</strong><br>'.format((yesterday))
#     res+='<strong>Confirmed Cases as of Today : {}</strong><br>\n'.format(today)
    res+='<strong>Data Source Used for Training: <br> <a href="https://prsindia.org/covid-19/cases"> Visit prsindia.org</a></strong></p>'
    res+='</div>\n'
    res+='<div style="flex-grow: 1; padding: 10px 10px 10px 10px;" class="temp_2">\n'
    res+=embbed_graph(region)
    res+=add_cases_compare(region, as_on, yesterday)
    res+='</div>\n'
    res+='</div>\n'
    res+='</div>\n'
    return res

CORS(app)

# main index page route
@app.route('/')
def home():
    return '<h1>API is working.. </h1>'

@app.route('/Home')
def Home():
    region = "India"
    forecast = forecast_cases(region)
    
    return result_to_html(forecast, region)


@app.route('/forecast',methods=['GET'])
def forecast():
    region = str(request.args["region"])
    forecast = forecast_cases(region)
    
    return result_to_html(forecast, region)

@app.route('/update')
def update():
    update_database()
    
    return "Database Updated"

@app.route('/train')
def train():
    fit_model()
    
    return "Database Updated"


if __name__ == "__main__":
    app.run(debug=True)


Overwriting app_v2.py


In [152]:
%%writefile website_elements_v2.py
import pandas as pd
import plotly.graph_objects as go

def embbed_graph(region):
    updatemenus = list([    dict(active=0, buttons=list([
                dict(label = 'Total',
                     method = 'update',
                     args = [{'visible': [True, True, True, True, True, True, True, False, False, False, False]},
                             {'title': 'COVID-19 Cases Forecast - Total Cases',
                              'annotations': ''}]),
                dict(label = 'Daywise',
                     method = 'update',
                     args = [{'visible': [False, False, False, False, False, False, False, True, True, True, True]},
                             {'title': 'COVID-19 Cases Forecast - New Cases',
                              'annotations': ''}]),        ]),
                x = 0.01,
                xanchor = 'left',
                y = 1.03,
                yanchor = 'top',    )  ])

    df = pd.read_csv("COVID_Database.csv")
    states_df = df.groupby('Region')
    df = pd.read_csv("COVID_Forecasted.csv")
    states_df2 = df.groupby('Region')
    #Create the graph
    fig = go.Figure()
    fig.add_trace(go.Scatter(x=pd.DataFrame(states_df2.get_group(region))["Date"][:-1], y=pd.DataFrame(states_df2.get_group(region))["Confirmed_Cases"], mode='lines+markers', name='Confirmed Cases'))
    fig.add_trace(go.Scatter(x=pd.DataFrame(states_df2.get_group(region))["Date"][:-1], y=pd.DataFrame(states_df2.get_group(region))["Active_Cases"], mode='lines', name='Active Cases'))
    fig.add_trace(go.Scatter(x=pd.DataFrame(states_df2.get_group(region))["Date"][:-1], y=pd.DataFrame(states_df2.get_group(region))["Cured/Discharged"], mode='lines', name='Cured/Discharged'))
    fig.add_trace(go.Scatter(x=pd.DataFrame(states_df2.get_group(region))["Date"][:-1], y=pd.DataFrame(states_df2.get_group(region))["Death"], mode='lines', name='Deaths'))
    fig.add_trace(go.Scatter(x=pd.DataFrame(states_df2.get_group(region))["Date"][:-1], y=pd.DataFrame(states_df2.get_group(region))["Forecasted"], mode='lines+markers', name='Forecasted Cases', line=dict(color='rgb(255, 100, 50)'), legendgroup="Forecasts"))
    fig.add_trace(go.Scatter(x=pd.DataFrame(states_df2.get_group(region))["Date"][:-1], y=pd.DataFrame(states_df2.get_group(region))["Forecasted_high"], mode='lines', name='Forecasted Cases', fill=None, line=dict(color='rgb(255, 150, 50)'), legendgroup="Forecasts", showlegend=False))
    fig.add_trace(go.Scatter(x=pd.DataFrame(states_df2.get_group(region))["Date"][:-1], y=pd.DataFrame(states_df2.get_group(region))["Forecasted_low"], mode='lines', name='Forecasted Cases', line=dict(color='rgb(255, 150, 50)'), fill='tonexty', legendgroup="Forecasts", showlegend=False))


    fig.add_trace(go.Scatter(x=pd.DataFrame(states_df2.get_group(region))["Date"][:-1], y=pd.DataFrame(states_df2.get_group(region))["Confirmed_Cases_Daywise"], mode='lines+markers', name='Confirmed Cases', visible=False))
    fig.add_trace(go.Scatter(x=pd.DataFrame(states_df2.get_group(region))["Date"][:-1], y=pd.DataFrame(states_df2.get_group(region))["Forecasted_Cases_Daywise"], mode='lines+markers', name='Forecasted Cases', line=dict(color='rgb(255, 100, 50)'), legendgroup="Forecasts", visible=False))
    fig.add_trace(go.Scatter(x=pd.DataFrame(states_df2.get_group(region))["Date"][:-1], y=pd.DataFrame(states_df2.get_group(region))["Forecasted_Cases_high_Daywise"], mode='lines', name='Forecasted Cases', fill=None, line=dict(color='rgb(255, 150, 50)'), legendgroup="Forecasts", showlegend=False, visible=False))
    fig.add_trace(go.Scatter(x=pd.DataFrame(states_df2.get_group(region))["Date"][:-1], y=pd.DataFrame(states_df2.get_group(region))["Forecasted_Cases_low_Daywise"], mode='lines', name='Forecasted Cases', line=dict(color='rgb(255, 150, 50)'), fill='tonexty', legendgroup="Forecasts", showlegend=False, visible=False))

    #fig.update_xaxes(tickangle=45, tickformat = '%d-%m')

    #Edit the layout
    fig.update_layout(template='plotly_white', legend=dict(x=0.01, y=0.95), margin=dict(l=20, r=20, t=50, b=20), hovermode='x unified', updatemenus=updatemenus)
    fig.update_layout(title='COVID-19 Cases Forecast - Total Cases', xaxis_title='Date', yaxis_title='Number of Cases')
    # fig.show()
    
    go_fig = fig.to_html(full_html=False)
    
    return go_fig


def get_menu():
    web_element = '''<head>
<meta name="viewport" content="width=device-width, initial-scale=1">
<link rel="stylesheet" href="https://cdnjs.cloudflare.com/ajax/libs/font-awesome/4.7.0/css/font-awesome.min.css">
<style>
/* Style the body */
body {
  font-family: Arial, Helvetica, sans-serif, Serif;
  margin: 0;
}

h1.Title {
  font-family: "Georgia, Times New Roman", Times, serif;
}

/* Header/Logo Title */
.header {
  padding: 0px;
  text-align: center;
  background: #333; //#1abc9c
  color: white;
  font-size: 20px;
  height: 15%;
}

/* Page Content */
.content {padding:20px;}

.navbar {
  overflow: hidden;
  background-color: #333;
  padding: 0px 0px 0px 0px;
  top: 70px;
  width: 100%;
}

.navbar a {
  float: left;
  font-size: 16px;
  color: white;
  text-align: center;
  padding: 14px 16px;
  text-decoration: none;
}

.dropdown {
  float: left;
  overflow: hidden;
}

.dropdown .dropbtn {
  font-size: 16px;  
  border: none;
  outline: none;
  color: white;
  padding: 14px 16px;
  background-color: inherit;
  font-family: inherit;
  margin: 0;
}

.navbar a:hover, .dropdown:hover .dropbtn {
  background-color: red;
}

.dropdown-content {
  display: none;
  position: absolute;
  background-color: #f9f9f9;
  min-width: 160px;
  box-shadow: 0px 8px 16px 0px rgba(0,0,0,0.2);
  z-index: 1;
}

.dropdown-content a {
  float: none;
  color: black;
  padding: 12px 16px;
  text-decoration: none;
  display: block;
  text-align: left;
}

.dropdown-content a:hover {
  background-color: #ddd;
}

.dropdown:hover .dropdown-content {
  display: block;
}

table {
  width:100%;
}
table, th, td {
  border: 1px solid black;
  border-collapse: collapse;
}
th, td {
  padding: 1px;
  text-align: left;
}
#t01 tr:nth-child(even) {
  background-color: #eee;
}
#t01 tr:nth-child(odd) {
 background-color: #fff;
}
#t01 th {
  background-color: #333;
  color: white;
}
</style>
</head>
<body>

<div class="header">
  <h1 class="Title" style="font:Times New Roman; padding: 20px 0px 0px 0px; color: white;"> COVID-19 Forecasts </h1>
  <div class="navbar">
  <a href="http://15.206.28.4:9000/Home">Home</a>
  <a href="http://15.206.28.4:9000/forecast?region=World">World</a>
  <div class="dropdown">
    <button class="dropbtn">States (A-H)
      <i class="fa fa-caret-down"></i>
    </button>
    <div class="dropdown-content">
    <a href="http://15.206.28.4:9000/forecast?region=Andaman and Nicobar Islands">Andaman and Nicobar Islands</a>
    <a href="http://15.206.28.4:9000/forecast?region=Andhra Pradesh">Andhra Pradesh</a>
    <a href="http://15.206.28.4:9000/forecast?region=Arunachal Pradesh">Arunachal Pradesh</a>
    <a href="http://15.206.28.4:9000/forecast?region=Assam">Assam</a>
    <a href="http://15.206.28.4:9000/forecast?region=Bihar">Bihar</a>
    <a href="http://15.206.28.4:9000/forecast?region=Chandigarh">Chandigarh</a>
    <a href="http://15.206.28.4:9000/forecast?region=Chhattisgarh">Chhattisgarh</a>
    <a href="http://15.206.28.4:9000/forecast?region=Dadra and Nagar Haveli and Daman and Diu">Dadra and Nagar Haveli and Daman and Diu</a>
    <a href="http://15.206.28.4:9000/forecast?region=Delhi">Delhi</a>
    <a href="http://15.206.28.4:9000/forecast?region=Goa">Goa</a>
    <a href="http://15.206.28.4:9000/forecast?region=Gujarat">Gujarat</a>
    <a href="http://15.206.28.4:9000/forecast?region=Haryana">Haryana</a>
    <a href="http://15.206.28.4:9000/forecast?region=Himachal Pradesh">Himachal Pradesh</a>
    </div>
    </div>
    <div class="dropdown">
    <button class="dropbtn">States (J-P)
      <i class="fa fa-caret-down"></i>
    </button>
    <div class="dropdown-content">
    <a href="http://15.206.28.4:9000/forecast?region=Jammu and Kashmir">Jammu and Kashmir</a>
    <a href="http://15.206.28.4:9000/forecast?region=Jharkhand">Jharkhand</a>
    <a href="http://15.206.28.4:9000/forecast?region=Karnataka">Karnataka</a>
    <a href="http://15.206.28.4:9000/forecast?region=Kerala">Kerala</a>
    <a href="http://15.206.28.4:9000/forecast?region=Ladakh">Ladakh</a>
    <a href="http://15.206.28.4:9000/forecast?region=Madhya Pradesh">Madhya Pradesh</a>
    <a href="http://15.206.28.4:9000/forecast?region=Maharashtra">Maharashtra</a>
    <a href="http://15.206.28.4:9000/forecast?region=Manipur">Manipur</a>
    <a href="http://15.206.28.4:9000/forecast?region=Meghalaya">Meghalaya</a>
    <a href="http://15.206.28.4:9000/forecast?region=Mizoram">Mizoram</a>
    <a href="http://15.206.28.4:9000/forecast?region=Nagaland">Nagaland</a>
    <a href="http://15.206.28.4:9000/forecast?region=Odisha">Odisha</a>
    <a href="http://15.206.28.4:9000/forecast?region=Puducherry">Puducherry</a>
    <a href="http://15.206.28.4:9000/forecast?region=Punjab">Punjab</a>
    </div>
    </div>
    <div class="dropdown">
    <button class="dropbtn">States (R-Z)
      <i class="fa fa-caret-down"></i>
    </button>
    <div class="dropdown-content">
    <a href="http://15.206.28.4:9000/forecast?region=Rajasthan">Rajasthan</a>
    <a href="http://15.206.28.4:9000/forecast?region=Sikkim">Sikkim</a>
    <a href="http://15.206.28.4:9000/forecast?region=Tamil Nadu">Tamil Nadu</a>
    <a href="http://15.206.28.4:9000/forecast?region=Telangana">Telangana</a>
    <a href="http://15.206.28.4:9000/forecast?region=Tripura">Tripura</a>
    <a href="http://15.206.28.4:9000/forecast?region=Uttar Pradesh">Uttar Pradesh</a>
    <a href="http://15.206.28.4:9000/forecast?region=Uttarakhand">Uttarakhand</a>
    <a href="http://15.206.28.4:9000/forecast?region=West Bengal">West Bengal</a>
    </div>
  </div> 
  </div>
</div>

</body>'''

    return web_element

def add_cases_compare(region, as_on, yesterday):
    df = pd.read_csv("Forecast.csv")
    yesterday_forecast = int(df[df.Region==region].iloc[0,-1])
    df = pd.read_csv("COVID_Forecasted.csv")
    df = df[df.Region==region]
    try:
        today_forecast = int(df.Forecasted[df.Date==as_on.strftime('%d-%m-%Y')].iloc[1])
    except:
        today_forecast = int(df.Forecasted[df.Date==as_on.strftime('%d-%m-%Y')].iloc[0])
    return '''<p style="font:Helvetica; color:#fff; font-size: 20px; position: absolute;  right: 10px;  top: 35px;  z-index: 1; text-align:right; line-height: 1.0; padding: 0px 10px 0px 0px "> Today's Forecast: <strong style="font-size: 24px;">{}</strong> <br> </p> 
    <p style="font:Helvetica; color:#fff;  position: absolute;  right: 10px;  top: 65px;  z-index: 1; text-align:right; line-height: 1.0;"> Our Forcasted Cases Yesterday: <strong style="font-size: 18px;">{}</strong> <br> Confirmed Cases Yesterday: <strong style="font-size: 16px;">{}</strong></p>'''.format(today_forecast, yesterday_forecast, int(yesterday))


Overwriting website_elements_v2.py


In [15]:
# %%writefile Update_Database_v2.py
from main_app_v2 import forecast_cases
import pandas as pd
import numpy as np
# from statsmodels.tsa.arima_model import ARIMA
# import matplotlib.pyplot as plt
# import itertools
import warnings
warnings.filterwarnings("ignore")

import requests
from bs4 import BeautifulSoup as bs
from datetime import date

def update(region_dict):
    res = requests.get("https://prsindia.org/covid-19/cases")
    soup = bs(res.content,'lxml')
    table = soup.find_all('table')[0] 
    df = pd.read_html(str(table))[0]
    df.drop(columns=["#"], inplace=True)
    cols = list(df.columns)
    cols[0] = "Region"
    cols = [i.replace(" ","_") for i in cols]
    df.columns = cols
    #df["Date"] = [date.today().strftime('%Y-%m-%d') for i in range(len(df))]
    #df.index = pd.to_datetime(df.Date,format='%Y-%m-%d')
    df["Date"] = [date.today().strftime('%d-%m-%Y') for i in range(len(df))]
    df.index = pd.to_datetime(df.Date,format='%d-%m-%Y')
    for i,region in enumerate(region_dict):
        region_dict[region] = region_dict[region].append(df[df.Region == region], sort=False, ignore_index=True)
        try:
            region_dict[region].index = pd.to_datetime(region_dict[region].Date,format='%d-%m-%Y')
        except:
            region_dict[region].index = pd.to_datetime(region_dict[region].Date,format='%Y-%m-%d')
        
    return region_dict

def update_database():
    
    df = pd.read_csv("COVID_Database.csv")
    cols = list(df.columns)
    cols = [i.replace(" ","_") for i in cols]
    df.columns = cols

    states_df = df.groupby('Region')
    state_df = [pd.DataFrame(states_df.get_group(state)) for state in states_df.groups]

    region_dict = {}
    for region in states_df.groups:
        region_dict[region] = pd.DataFrame(states_df.get_group(region))
        try:
            region_dict[region].index = pd.to_datetime(region_dict[region].Date,format='%Y-%m-%d')
        except:
            region_dict[region].index = pd.to_datetime(region_dict[region].Date,format='%d-%m-%Y')

        region_dict = update(region_dict)
    #     break

    df = pd.DataFrame()
    for region in region_dict.keys():
        df = df.append(region_dict[region], sort=False, ignore_index=True)

    cols = list(df.columns)
    cols = [i.replace(" ","_") for i in cols]
    df.columns = cols
    df.drop_duplicates(inplace=True)
    
    df.to_csv("COVID_Database.csv", index=False)
    
    forecasted_database()
    
    return "Database Updated"


def forecasted_database():    
    df = pd.read_csv("COVID_Database.csv")
    df["Forecasted"] = ["" for i in range(len(df))]
    regions = sorted(list(df.Region.value_counts().keys()))

    df_old = pd.read_csv("Forecast.csv")
    
    forecast_df = pd.DataFrame(columns=['Date', 'Region', 'Confirmed_Cases', 'Active_Cases', 'Cured/Discharged',
           'Death', 'Forecasted', 'Forecasted_high', 'Forecasted_low', 'Confirmed_Cases_Daywise'])
    
    forecasts = []
    forecasts_high = []
    forecasts_low = []
    forecasts_daywise = []
    forecasts_daywise_high = []
    forecasts_daywise_low = []
    days = []
    region_x = []
#     forecasted = []
#     forecasted_region = []
    n=31
    for region in regions:
        forecast,std_error,confidence_interval = forecast_cases(region, n=n, return_all=True)
        
        forecast_old = df_old.Forecasted_Cases[df_old.Region==region].iloc[0]
        forecast_old = forecast[0]-forecast_old
        forecasts_daywise.append(forecast_old)
        forecasts_daywise_high.append(forecast_old+(forecast_old*0.05))
        forecasts_daywise_low.append(forecast_old-(forecast_old*0.05))
        
        forecast_high = confidence_interval[:,1]
        forecast_low = confidence_interval[:,0]
        forecasts.extend(forecast)
        forecasts_high.extend(forecast_high)
        forecasts_low.extend(forecast_low)
        forecasts_daywise.extend(pd.Series(forecast).diff(periods=1).iloc[1:])
        forecasts_daywise_high.extend(pd.Series(forecast_high).diff(periods=1).iloc[1:])
        forecasts_daywise_low.extend(pd.Series(forecast_low).diff(periods=1).iloc[1:])
        days.extend(pd.date_range(date.today(), periods=n).strftime('%d-%m-%Y'))
        region_x.extend([region for i in range(n)])
#         forecasted.append(forecast[0])
#         forecasted_region.append(region)

    forecast_df.Forecasted = forecasts
    forecast_df.Date = days
    forecast_df.Region = region_x
    forecast_df.Forecasted_high = forecasts_high
    forecast_df.Forecasted_low = forecasts_low
    forecast_df["Forecasted_Cases_Daywise"] = forecasts_daywise
    forecast_df["Forecasted_Cases_high_Daywise"] = forecasts_daywise_high
    forecast_df["Forecasted_Cases_low_Daywise"] = forecasts_daywise_low
    forecast_df.index = pd.to_datetime(forecast_df.Date)

    df2 = pd.concat([df,forecast_df], join='outer', ignore_index=True)
    df2 = df2.groupby("Region", as_index=False).apply(lambda x: x.iloc[:-1])
    df2.index = df2.Date 
    
    
    cols = ['Confirmed_Cases', 'Active_Cases', 'Cured/Discharged', 'Death']
    for col in cols:
        df2[col] = df2[col].shift(-1)
    df2 = df2.dropna(how="all",subset=['Confirmed_Cases', 'Active_Cases','Forecasted'])

    for region in regions:
        df2.Confirmed_Cases_Daywise[df2.Region==region] = df2.Confirmed_Cases[df2.Region==region].diff(periods=1)

    df2.to_csv("COVID_Forecasted.csv", index=False)

In [165]:
def keep_in_memory():
    today = date.today().strftime('%d-%m-%Y')
    df = pd.read_csv("COVID_Database.csv")
    df = df[df.Date==today]
    if len(df) > 0:
        return
    df = pd.read_csv("Covid_Forecasted.csv")
    df = df[df.Date==today]
    regions = list(df.Region)
    forecasted = list(df.Forecasted)
    df = pd.DataFrame(columns=["Region", "Forecasted_Cases"])
    df.Region = regions
    df.Forecasted_Cases = forecasted
    df.Forecasted_Cases.bfill(inplace=True)
    df.drop_duplicates(inplace=True)
    df.to_csv("Forecast.csv", index=False)

In [24]:
update_database()

'Database Updated'

In [25]:
fit_model()

'Please Update Database'

In [22]:
start = time.time()
fit_model()
end = time.time()
print("Time Taken: {}s".format(end-start))

Time Taken: 0.014996767044067383s


with open("temp.html", "w") as file:
    for row in res:
        file.write(row)

In [ ]:
df = pd.read_csv(r"F:\DataScience\AWS\COVID_Database.csv")
df.tail()

In [ ]:
states_df = df.groupby('Region')
state_df = [pd.DataFrame(states_df.get_group(state)) for state in states_df.groups]

In [ ]:
n = 30
region_id = 15
state = state_df[region_id].copy()
region = state.Region.iloc[0]
state["Confirmed_Cases_diff"] = state.Confirmed_Cases.diff(n)[n:]
state.index = state.Date
state.tail(9)

In [ ]:
forecast, std_error, confidence_interval = forecast_cases(region,return_all=True)

In [ ]:
forecast

In [ ]:
std_error

In [ ]:
confidence_interval[:5]

In [ ]:
confidence_interval1 = []
for i, error in enumerate(std_error):
    confidence_interval1.append([confidence_interval[i,0]-error,confidence_interval[i,1]+error])
    
confidence_interval1 = np.array(confidence_interval1)
confidence_interval1

In [ ]:
n=7
m=1 #2
forecasted_difference = pd.DataFrame(index=pd.date_range(state.Date[-m], periods=n)) 
forecasted_difference["Confirmed_Cases"] = state.Confirmed_Cases[-m:]
forecasted_difference["Confirmed_Cases"][:m] = state.Confirmed_Cases[-m:]
#pd.DataFrame(index=state.Date[-n:])
forecasted_difference["Confirmed_Cases_diff"] = state.Confirmed_Cases_diff[-m:]
forecasted_difference["Confirmed_Cases_diff"][:m] = state.Confirmed_Cases_diff[-m:]
forecasted_difference["forecasted"] = forecast

forecasted_high = pd.DataFrame([i*1.05 for i in forecasted_difference.forecasted], index=pd.date_range(state.Date[-m], periods=n))
forecasted_low = pd.DataFrame([i*0.95 for i in forecasted_difference.forecasted], index=pd.date_range(state.Date[-m], periods=n))

# forecasted_difference["forecasted_high"] = forecasted_high
# forecasted_difference["forecasted_low"] = forecasted_low

forecasted_difference["forecasted_high"] = confidence_interval1[:,0]
forecasted_difference["forecasted_low"] = confidence_interval1[:,1]

In [ ]:
forecasted_difference

In [ ]:
plt.figure("Forecast", figsize=(16,8))
plt.plot(forecasted_difference.index, forecasted_difference.Confirmed_Cases, linewidth=2.0) #, color="blue")
plt.plot(forecasted_difference.index, forecasted_difference.forecasted) #, color="orange")
# plt.plot(forecasted_difference.index, forecasted_difference.Confirmed_Cases, linewidth=1.0) #, color="blue")
plt.plot(forecasted_difference.index, forecasted_difference.forecasted_high, color="dimgray", linestyle="--")
plt.plot(forecasted_difference.index, forecasted_difference.forecasted_low, color="dimgray", linestyle="--")
plt.legend()

In [ ]:
import plotly.graph_objects as go
import chart_studio.tools as tls
import json
from plotly.utils import PlotlyJSONEncoder

In [ ]:
#Create the graph
data = [go.Scatter(x=state_df[15]["Date"], y=state_df[15]["Confirmed_Cases"], mode='lines+markers', name='Confirmed Cases'),
go.Scatter(x=state_df[15]["Date"], y=state_df[15]["Active_Cases"], mode='lines+markers', name='Active Cases'),
go.Scatter(x=state_df[15]["Date"], y=state_df[15]["Cured/Discharged"], mode='lines+markers', name='Cured/Discharged'),
go.Scatter(x=state_df[15]["Date"], y=state_df[15]["Death"], mode='lines+markers', name='Deaths')]

fig = go.Figure(data)

#Edit the layout
fig.update_layout(template='plotly_white', legend=dict(x=0.01, y=0.99))
fig.update_layout(title='COVID-19 Cases', xaxis_title='Date', yaxis_title='Number of Cases')
fig.show()

graphJSON = json.dumps(data, cls=PlotlyJSONEncoder) #plotly.utils.PlotlyJSONEncoder)

In [ ]:
@app.route('/')
def index():

    bar = create_plot()
    return render_template('index.html', plot=bar)

In [ ]:
go_fig = fig.to_html(full_html=False)

In [ ]:
def get_graph():
    #Create the graph
    data = [go.Scatter(x=state_df[15]["Date"], y=state_df[15]["Confirmed_Cases"], mode='lines+markers', name='Confirmed Cases'),
    go.Scatter(x=state_df[15]["Date"], y=state_df[15]["Active_Cases"], mode='lines+markers', name='Active Cases'),
    go.Scatter(x=state_df[15]["Date"], y=state_df[15]["Cured/Discharged"], mode='lines+markers', name='Cured/Discharged'),
    go.Scatter(x=state_df[15]["Date"], y=state_df[15]["Death"], mode='lines+markers', name='Deaths')]

    fig = go.Figure(data)

    #Edit the layout
    fig.update_layout(template='plotly_white', legend=dict(x=0.01, y=0.99))
    fig.update_layout(title='COVID-19 Cases', xaxis_title='Date', yaxis_title='Number of Cases')
    
    go_fig = fig.to_html(full_html=False)
    
    return go_fig 

In [ ]:
df.head()

In [ ]:
df = pd.read_csv(r"F:\DataScience\AWS\COVID_Database.csv")
df["Forecasted"] = ["" for i in range(len(df))]
regions = sorted(list(df.Region.value_counts().keys()))

forecast_df = pd.DataFrame(columns=['Date', 'Region', 'Confirmed_Cases', 'Active_Cases', 'Cured/Discharged',
       'Death', 'Forecasted', 'Forecasted_high', 'Forecasted_low', 'Confirmed_Cases_Daywise'])

forecasts = []
forecasts_high = []
forecasts_low = []
forecasts_daywise = []
forecasts_daywise_high = []
forecasts_daywise_low = []
days = []
region_x = []
forecasted = []
forecasted_region = []
n=30
for region in regions:
    forecast,std_error,confidence_interval = forecast_cases(region, n=n, return_all=True)
    forecast_high = confidence_interval[:,1]
    forecast_low = confidence_interval[:,0]
    forecasts.extend(forecast)
    forecasts_high.extend(forecast_high)
    forecasts_low.extend(forecast_low)
    forecasts_daywise.extend(pd.Series(forecast).diff(periods=1))
    forecasts_daywise_high.extend(pd.Series(forecast_high).diff(periods=1))
    forecasts_daywise_low.extend(pd.Series(forecast_low).diff(periods=1))
    days.extend(pd.date_range(date.today(), periods=n).strftime('%d-%m-%Y'))
    region_x.extend([region for i in range(n)])
    forecasted.append(forecast[0])
    forecasted_region.append(region)
    
forecast_df.Forecasted = forecasts
forecast_df.Date = days
forecast_df.Region = region_x
forecast_df.Forecasted_high = forecasts_high
forecast_df.Forecasted_low = forecasts_low
forecast_df["Forecasted_Cases_Daywise"] = forecasts_daywise
forecast_df["Forecasted_Cases_high_Daywise"] = forecasts_daywise_high
forecast_df["Forecasted_Cases_low_Daywise"] = forecasts_daywise_low
forecast_df.index = pd.to_datetime(forecast_df.Date)

df2 = pd.concat([df,forecast_df], join='outer', ignore_index=True)
df2 = df2.groupby("Region", as_index=False).apply(lambda x: x.iloc[:-1])
df2.index = df2.Date

for region in regions:
    df2.Confirmed_Cases_Daywise[df2.Region==region] = df2.Confirmed_Cases[df2.Region==region].diff(periods=1)

df2.to_csv("COVID_Forecasted.csv")

df3 = pd.DataFrame(columns=["Region", "Forecasted_Cases"])
df3.Region = forecasted_region
df3.Forecasted_Cases = forecasted
df3.to_csv("Forecast.csv")

In [ ]:
states_df = df2.groupby('Region')
state_df = [pd.DataFrame(states_df.get_group(state)) for state in states_df.groups]

In [ ]:
region="Karnataka"

In [ ]:
updatemenus = list([    dict(active=0, buttons=list([
            dict(label = 'Total',
                 method = 'update',
                 args = [{'visible': [True, True, True, True, True, True, True, False, False, False, False]},
                         {'title': 'COVID-19 Cases Forecast - Total Cases',
                          'annotations': ''}]),
            dict(label = 'Daywise',
                 method = 'update',
                 args = [{'visible': [False, False, False, False, False, False, False, True, True, True, True]},
                         {'title': 'COVID-19 Cases Forecast - New Cases',
                          'annotations': ''}]),        ]),
            x = 0.01,
            xanchor = 'left',
            y = 1.05,
            yanchor = 'top',    )  ])

df = pd.read_csv("COVID_Database.csv")
states_df = df.groupby('Region')
df = pd.read_csv("COVID_Forecasted.csv")
states_df2 = df.groupby('Region')
#Create the graph
fig = go.Figure()
fig.add_trace(go.Scatter(x=pd.DataFrame(states_df2.get_group(region))["Date"], y=pd.DataFrame(states_df.get_group(region))["Confirmed_Cases"], mode='lines+markers', name='Confirmed Cases'))
fig.add_trace(go.Scatter(x=pd.DataFrame(states_df2.get_group(region))["Date"], y=pd.DataFrame(states_df.get_group(region))["Active_Cases"], mode='lines', name='Active Cases'))
fig.add_trace(go.Scatter(x=pd.DataFrame(states_df2.get_group(region))["Date"], y=pd.DataFrame(states_df.get_group(region))["Cured/Discharged"], mode='lines', name='Cured/Discharged'))
fig.add_trace(go.Scatter(x=pd.DataFrame(states_df2.get_group(region))["Date"], y=pd.DataFrame(states_df.get_group(region))["Death"], mode='lines', name='Deaths'))
fig.add_trace(go.Scatter(x=pd.DataFrame(states_df2.get_group(region))["Date"], y=pd.DataFrame(states_df2.get_group(region))["Forecasted"], mode='lines+markers', name='Forecasted Cases', line=dict(color='rgb(255, 100, 50)'), legendgroup="Forecasts"))
fig.add_trace(go.Scatter(x=pd.DataFrame(states_df2.get_group(region))["Date"], y=pd.DataFrame(states_df2.get_group(region))["Forecasted_high"], mode='lines', name='Forecasted Cases', fill=None, line=dict(color='rgb(255, 150, 50)'), legendgroup="Forecasts", showlegend=False))
fig.add_trace(go.Scatter(x=pd.DataFrame(states_df2.get_group(region))["Date"], y=pd.DataFrame(states_df2.get_group(region))["Forecasted_low"], mode='lines', name='Forecasted Cases', line=dict(color='rgb(255, 150, 50)'), fill='tonexty', legendgroup="Forecasts", showlegend=False))


fig.add_trace(go.Scatter(x=pd.DataFrame(states_df2.get_group(region))["Date"], y=pd.DataFrame(states_df2.get_group(region))["Confirmed_Cases_Daywise"], mode='lines+markers', name='Confirmed Cases', visible=False))
fig.add_trace(go.Scatter(x=pd.DataFrame(states_df2.get_group(region))["Date"], y=pd.DataFrame(states_df2.get_group(region))["Forecasted_Cases_Daywise"], mode='lines+markers', name='Forecasted Cases', line=dict(color='rgb(255, 100, 50)'), legendgroup="Forecasts", visible=False))
fig.add_trace(go.Scatter(x=pd.DataFrame(states_df2.get_group(region))["Date"], y=pd.DataFrame(states_df2.get_group(region))["Forecasted_Cases_high_Daywise"], mode='lines', name='Forecasted Cases', fill=None, line=dict(color='rgb(255, 150, 50)'), legendgroup="Forecasts", showlegend=False, visible=False))
fig.add_trace(go.Scatter(x=pd.DataFrame(states_df2.get_group(region))["Date"], y=pd.DataFrame(states_df2.get_group(region))["Forecasted_Cases_low_Daywise"], mode='lines', name='Forecasted Cases', line=dict(color='rgb(255, 150, 50)'), fill='tonexty', legendgroup="Forecasts", showlegend=False, visible=False))

#fig.update_xaxes(tickangle=45, tickformat = '%d-%m')

#Edit the layout
fig.update_layout(template='plotly_white', legend=dict(x=0.01, y=0.95), margin=dict(l=20, r=20, t=50, b=20), hovermode='x unified', updatemenus=updatemenus)
fig.update_layout(title='COVID-19 Cases Forecast - Total Cases', xaxis_title='Date', yaxis_title='Number of Cases')
fig.show()

In [73]:

import pandas as pd
import numpy as np
from statsmodels.tsa.arima_model import ARIMA
import matplotlib.pyplot as plt
import itertools
import warnings
warnings.filterwarnings("ignore")

import requests
from bs4 import BeautifulSoup as bs
from datetime import date, timedelta
import joblib


def get_previous_cases(region):
    df = pd.read_csv("COVID_Database.csv")
    df.index = df.Date
    try:
        yesterday = df.Confirmed_Cases[df.Region==region][str(date.today().strftime('%d-%m-%Y'))]
        as_on = str(date.today())
    except:
        yesterday = df.Confirmed_Cases[df.Region==region][-1]
        as_on = df.Date[df.Region==region][-1]
    
    del df
    return yesterday, as_on

def find_best_params(state, pdq):
    aic_values = []
    for param in pdq:
        try:
            #initilizing ARIMA model
            model = ARIMA(state.Confirmed_Cases_diff[:-2], order=param)
            model_fit = model.fit()
#             print(param, model_fit.aic)
            aic_values.append([param, model_fit.aic])
        except:
            pass

    aic_values = np.array(aic_values)
    a = np.where(aic_values == aic_values[:,1].min())[0][0]
    return aic_values[a][0]

def fit_model():
    df = pd.read_csv("COVID_Database.csv")
    cols = list(df.columns)
    cols = [i.replace(" ","_") for i in cols]
    df.columns = cols

    states_df = df.groupby('Region')
    state_df = [pd.DataFrame(states_df.get_group(state)) for state in states_df.groups]
    
    p=range(0,9)
    d=range(0,3)
    q=range(0,5)
    pdq = list(itertools.product(p,d,q))

    state_model = {}
    for state in state_df[:]:
        
        try:
            state.index = pd.to_datetime(state.Date,format='%Y-%m-%d')
        except:
            state.index = pd.to_datetime(state.Date,format='%d-%m-%Y')
    #     state.drop(columns=["Date","Region"], axis=1, inplace=True)

        print("Processing:", state.Region[0])

        state.Confirmed_Cases_diff = state.Confirmed_Cases
        params = find_best_params(state, pdq)

        #initilizing ARIMA model
        model = ARIMA(state.Confirmed_Cases_diff[:-2], order=params)
        model_fit = model.fit()
#         print(params)

        state_model[str(state.Region[0])] = model_fit

#         break

    for region in state_model.keys():
        joblib.dump(state_model[region], region+".ml_old")
        
#     print("DONE")
    return "DONE"

def forecast_cases(region, n=7, return_all=False):
    
    model_fit = joblib.load(region+".ml_old")
    # multi-step out-of-sample forecast
    forecast, std_error, confidence_interval = model_fit.forecast(steps=n)
    forecast = [int(round(cases)) for cases in forecast]
    if return_all:
        return forecast, std_error, confidence_interval
    else:
        return forecast


In [74]:
fit_model()

Processing: Andaman and Nicobar Islands
Processing: Andhra Pradesh
Processing: Arunachal Pradesh
Processing: Assam
Processing: Bihar
Processing: Chandigarh
Processing: Chhattisgarh
Processing: Dadra and Nagar Haveli and Daman and Diu
Processing: Delhi
Processing: Goa
Processing: Gujarat
Processing: Haryana
Processing: Himachal Pradesh
Processing: India
Processing: Jammu and Kashmir
Processing: Jharkhand
Processing: Karnataka
Processing: Kerala
Processing: Ladakh
Processing: Madhya Pradesh
Processing: Maharashtra
Processing: Manipur
Processing: Meghalaya
Processing: Mizoram
Processing: Nagaland
Processing: Odisha
Processing: Puducherry
Processing: Punjab
Processing: Rajasthan
Processing: Sikkim
Processing: State assignment pending
Processing: Tamil Nadu
Processing: Telangana
Processing: Tripura
Processing: Uttar Pradesh
Processing: Uttarakhand
Processing: West Bengal
Processing: World


'DONE'

In [96]:
forecasts=[]
for region in regions:
    forecasts.append(forecast_cases(region)[1])

df = pd.DataFrame()
df["Region"] = regions
df["Forecasted_Cases"] = forecasts
df.to_csv("Forecast.csv", index=False)
df.head()

,Region,Forecasted_Cases
0,Andaman and Nicobar Islands,410
1,Andhra Pradesh,125593
2,Arunachal Pradesh,1471
3,Assam,37156
4,Bihar,49428


In [11]:
region = "Karnataka"
df_old = pd.read_csv("Forecast.csv")
forecast_old = df_old.Forecasted_Cases[df_old.Region==region].iloc[0]

In [12]:
forecast_old

119777